In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/NEU_DET_SurfaceAnomalyDetection

/content/drive/MyDrive/NEU_DET_SurfaceAnomalyDetection


In [3]:
!pip install pennylane
#!pip install pennylane-lightning[gpu]
#!pip install wheel cuquantum

In [4]:
##############################################
'''
image_folder = 'IMAGES'
annot_folder = 'ANNOTATIONS'
noDefect_folder = 'NO_DEFECT'
'''
##############################################
resizedImage_folder = 'ReIMAGES'
resizedNoDefectImage_folder = 'ReNO_DEFECT'
q_feature_file_path = 'Q_Feature_Maps/ALL_Q_featureMap_4x32x32.pt'
q_feature_labels_file_path = 'Q_Feature_Maps/ALL_Q_featureMap_4x32x32_labels.pt'

In [5]:
# Input dimension
#input_dim = (64, 64)
input_dim = (200, 200)

In [6]:
# Test accuracy function
def test_acc(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images) #-1x2
            predictions = np.argmax(outputs.detach().cpu().numpy(), axis = 1)
            correct += np.sum(np.logical_not(np.logical_xor(predictions, labels.numpy())))
            total += len(predictions)

    accuracy = correct / total
    return accuracy

## Preparing dataset. Resize to fit the criterion for applying Quanvolution

In [7]:
import os
import cv2
import numpy as np
selected_anomalies = ["rolled-in", "patches", "inclusion", "scratches"]
labels_ano = []
X_ano = []
for fname in os.listdir(resizedImage_folder):
    for i in range(len(selected_anomalies)):
        if(fname.split('_')[0] == selected_anomalies[i]):
            labels_ano.append(i+1)
            break
    fpath = os.path.join(resizedImage_folder, fname)
    im = cv2.imread(fpath, 0) # gray scale read
    im = cv2.resize(im, input_dim, interpolation = cv2.INTER_LINEAR)
    im = im.reshape(1, im.shape[0], im.shape[1])
    X_ano.append(im)
X_ano = np.array(X_ano)
labels_ano = np.array(labels_ano)

In [8]:
X_norm = []
labels_norm = np.zeros(len(os.listdir(resizedNoDefectImage_folder)))
for fname in os.listdir(resizedNoDefectImage_folder):
    fpath = os.path.join(resizedNoDefectImage_folder, fname)
    im = cv2.imread(fpath, 0) # gray scale read
    im = cv2.resize(im, input_dim, interpolation = cv2.INTER_LINEAR)
    im = im.reshape(1, im.shape[0], im.shape[1])
    X_norm.append(im)
X_norm = np.array(X_norm)

In [9]:
print(f"Shapes\nX_anomaly: {X_ano.shape}, labels_anomaly: {labels_ano.shape}\nX_normal: {X_norm.shape}, labels_normal: {labels_norm.shape}")

Shapes
X_anomaly: (1103, 1, 200, 200), labels_anomaly: (1103,)
X_normal: (1927, 1, 200, 200), labels_normal: (1927,)


## BINARY classification

In [10]:
# Selecting equanll number of normal images
random_idx = np.random.randint(low=0, high=X_norm.shape[0], size = (X_ano.shape[0]))
X_norm = X_norm[random_idx]
labels_norm = labels_norm[random_idx]
X_norm.shape

(1103, 1, 200, 200)

In [11]:
X = np.concatenate((X_norm, X_ano), axis = 0)
Y_poly = np.concatenate((labels_norm, labels_ano), axis = 0)
Y_bin = Y_poly
Y_bin [Y_bin > 0] = 1
X.shape, Y_bin.shape

((2206, 1, 200, 200), (2206,))

In [12]:
# Normalize
X = X/255.0

## No need to run if performing a k-fold cross validation

In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y_bin, test_size=0.25, random_state = 42)

In [ ]:
y_train = np.array([np.eye(2)[int(i),:] for i in y_train])

In [ ]:
X_train.shape, y_train.shape

((1654, 1, 200, 200), (1654, 2))

In [ ]:
#import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
#from pennylane import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

## Testing a classical Model

In [ ]:
from Models import ClassicalModel2

In [ ]:
from Models import ClassicalModel2
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm


# load the quantum
#Q_featureMap = torch.load(q_feature_file_path)
#Q_labels = torch.load(q_feature_labels_file_path)

# Model to test
#best_hybrid_model = hybridModel
criterion = torch.nn.CrossEntropyLoss()  # For classification tasks, adjust accordingly
num_epochs = 200

# Convert your features and labels to PyTorch tensors


# Define KFold cross validator
kf = KFold(n_splits=6, shuffle=True, random_state=42)
results_dict = {}
models = {}

# Define a device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ClassicalModel2()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0008)

# Split the data into training and validation sets for this fold
X_train, X_val, y_train, y_val = train_test_split(X, Y_bin, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_train = torch.tensor(np.array(np.eye(2))[y_train.to(torch.int)], dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
# Convert to PyTorch DataLoader
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64)

# Training loop for this fold
for epoch in range(num_epochs):  # num_epochs is a variable you should define
    total_loss = 0
    avg_loss = 0
    for batch_idx, (feature, label) in enumerate(train_loader):
        feature, label = feature.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(feature)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if(epoch%5 == 0):
        avg_loss = total_loss / len(train_loader.dataset)
        test_accuracy = test_acc(model, val_loader, device)
        print(f"test_acc: {test_accuracy}, loss: {avg_loss}")



test_acc: 0.669683257918552, loss: 0.010681362615691291
test_acc: 0.8190045248868778, loss: 0.006562039965674991
test_acc: 0.8800904977375565, loss: 0.004489176338559646
test_acc: 0.9004524886877828, loss: 0.0030860319192987993
test_acc: 0.9230769230769231, loss: 0.002409072339433391
test_acc: 0.9253393665158371, loss: 0.002303132673307341
test_acc: 0.9276018099547512, loss: 0.0016023221208192324
test_acc: 0.9434389140271493, loss: 0.001362574841117575
test_acc: 0.9321266968325792, loss: 0.000947341572624249
test_acc: 0.9434389140271493, loss: 0.001665273662276414
test_acc: 0.9502262443438914, loss: 0.0004408901648135547
test_acc: 0.9660633484162896, loss: 0.00031334441656757846
test_acc: 0.920814479638009, loss: 0.0011968617807955495
test_acc: 0.9479638009049773, loss: 9.90864943133249e-05
test_acc: 0.9502262443438914, loss: 8.80892754663331e-05
test_acc: 0.9411764705882353, loss: 6.824563923748548e-05
test_acc: 0.9479638009049773, loss: 9.911918856488391e-06
test_acc: 0.9479638009049

### Quanvolution Model

This layer will come after one classical conv layer with output channel = 1. So instead of throwing 64x64 images in the quanv layer, we shall down-size them to 21x21, by applying kernel of size 4x4, stride = 3, output channel =1

In [ ]:
n_qubits = 9
n_layers = 3
weight_shapes = {"weights": (n_layers, n_qubits, 3)}
dev = qml.device("default.qubit", wires = n_qubits)
@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    # Hadamard Layer
    for wire in range(n_qubits):
        qml.Hadamard(wires = wire)
    # Embedding layer
    qml.AngleEmbedding(inputs, wires=range(n_qubits))

    # Variational layer
    for _ in range(n_layers):
        qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

qlayer = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)

In [ ]:
n_qubits = 9
n_layers = 3


def q_conv(input, kernel_size, n_layers, stride):
    n_qubits = int(kernel_size**2)

    # Assert the rules
    assert len(input.shape) == 4                                                # (batch_size, n_channels, x_dim, y_dim)
    assert input.shape[-1] == input.shape[-2]
    assert stride >= 0
    assert input.shape[1] == 1                                                  # Supports only one channel only

    # First define a q-node
    weight_shapes = {"weights": (n_layers, n_qubits, 3)}
    dev = qml.device("default.qubit", wires = n_qubits)
    @qml.qnode(dev, interface="torch")
    def quantum_circuit(inputs, weights):
        # Hadamard Layer
        for wire in range(n_qubits):
            qml.Hadamard(wires = wire)
        # Embedding layer
        qml.AngleEmbedding(inputs, wires=range(n_qubits))

        # Variational layer
        for _ in range(n_layers):
            qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))

        return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

    qlayer = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)

    k = kernel_size                                                             # kernel dimension
    output = None
    iterator = 0
    output_shape = int((input.shape[-1]-k)/stride + 1)
    print(f"output_shape = {output_shape}\n")
    for i in range(0, input.shape[-2], stride):
        if(i+k > input.shape[-2]):
            break
        for j in range(0, input.shape[-1], stride):
            if(j+k > input.shape[-1]):
                break
            x = torch.flatten(input[:,:, i:i+k, j:j+k], start_dim=1)
            exp_vals = qlayer(x).view(input.shape[0], n_qubits, 1)
            if(iterator>0):
                output = torch.cat((output, exp_vals), -1)
            else:
                output = exp_vals
            iterator +=1
    output
    return output

In [ ]:
X = torch.tensor(range(1, 49))
X = torch.reshape(X, (3,1,4,4))
#print(x)
stride = 2
k = 2
input = X
iterator = 0
output = None
for i in range(0, input.shape[-2], stride):
    if(i+k > input.shape[-2]):
        break
    for j in range(0, input.shape[-1], stride):
        if(j+k > input.shape[-1]):
            break
        x = torch.flatten(input[:,:, i:i+k, j:j+k], start_dim=1)
        print(x.shape, qlayer(x).shape)
        exp_vals = qlayer(x).view(input.shape[0], n_qubits, 1)
        if(iterator>0):
            output = torch.cat((output, exp_vals), -1)
        else:
            output = exp_vals
        iterator +=1

torch.Size([3, 4]) torch.Size([3, 9])
torch.Size([3, 4]) torch.Size([3, 9])
torch.Size([3, 4]) torch.Size([3, 9])
torch.Size([3, 4]) torch.Size([3, 9])


## Define a class for using Q_conv

In [ ]:
class Q_conv:
    def __init__(self, kernel_size, n_layers, stride):
        self.input = input
        self.kernel_size = kernel_size
        self.n_layers = n_layers
        self.stride = stride
        self.qlayer = None


    def q_conv(self, input):
        n_qubits = int(self.kernel_size**2)

        # Assert the rules
        assert len(input.shape) == 4                                                # (batch_size, n_channels, x_dim, y_dim)
        assert input.shape[-1] == input.shape[-2]
        assert self.stride >= 0
        assert input.shape[1] == 1                                                  # Supports only one channel only

        # First define a q-node
        weight_shapes = {"weights": (self.n_layers, n_qubits, 3)}
        #dev = qml.device("lightning.gpu", wires = n_qubits) if torch.cuda.is_available() else qml.device("default.qubit", wires = n_qubits)
        #dev = qml.device("default.qubit", wires = n_qubits) if device.type=='cpu' else qml.device("lightning.gpu", wires = n_qubits)
        dev = qml.device("default.qubit", wires = n_qubits)

        @qml.qnode(dev, interface="torch")
        def quantum_circuit(inputs, weights):
            #print(f"#################weights = {weights}#################")
            '''
            # Hadamard Layer
            for wire in range(n_qubits):
                qml.Hadamard(wires = wire)
            '''
            # Embedding layer
            qml.AngleEmbedding(inputs, wires=range(n_qubits))

            # Variational layer
            for _ in range(self.n_layers):
                #print(weights.is_cuda)

                qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
                #print("Test")

            return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

        self.qlayer = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)

        k = self.kernel_size                                                             # kernel dimension
        output = None
        iterator = 0
        output_shape = int((input.shape[-1]-k)/self.stride + 1)
        #print(f"output_shape = {output_shape}\n")
        for i in range(0, input.shape[-2], self.stride):
            if(i+k > input.shape[-2]):
                break
            for j in range(0, input.shape[-1], self.stride):
                if(j+k > input.shape[-1]):
                    break
                x = torch.flatten(input[:,:, i:i+k, j:j+k], start_dim=1)
                exp_vals = self.qlayer(x).view(input.shape[0], n_qubits, 1)
                if(iterator>0):
                    output = torch.cat((output, exp_vals), -1)
                else:
                    output = exp_vals
                iterator +=1
        output = torch.reshape(output, (output.shape[0], output.shape[1], output_shape, output_shape))
        return output
    def __call__(self, input):
        return self.q_conv(input)

In [ ]:
x = torch.rand(1, 1, 64, 64)
q_conv = Q_conv(2, 3, 2)

import time
start = time.time()
print(q_conv(input = x).shape)

print(time.time()-start)

torch.Size([1, 4, 32, 32])
83.35161471366882


In [ ]:
!pip install pennylane-lightning[gpu]

In [ ]:
!pip install wheel cuquantum

## Test Q_conv

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pennylane as qml
from pennylane import numpy as np
from tqdm import tqdm

# Set Up the Environment
# device is defined as an Universal variable

# Data Preparation (Assuming data is pre-loaded)
features = X_train
labels = y_train

# Convert data to PyTorch tensors and move them to the device (GPU if available)
features = torch.tensor(features, dtype=torch.float32).to(device)
labels = torch.tensor(labels, dtype=torch.float32).to(device)

# Create datasets and dataloaders
batch_size = 20  # Define your batch size
train_dataset = TensorDataset(features, labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create the Hybrid Model (Same as before)
class HybridNN(nn.Module):
    def __init__(self):
        super(HybridNN, self).__init__()
        # i/p_shape: (-1, 1, 32, 32)
        self.conv1 = nn.Conv2d(1, 1, kernel_size = 4, stride = 1)               # o/p_shape: (-1, 1 , 29, 29)

        self.maxpool1 = nn.MaxPool2d(kernel_size = 4, stride = 2)               # o/p_shape: (-1, 1 , 13, 13)

        self.q_conv1 = Q_conv(kernel_size = 2, n_layers = 4, stride = 1)        # o/p_shape: (-1, 4,  12,  12)

        self.conv2 = nn.Conv2d(4, 36, kernel_size=3, stride = 2)                # o/p_shape: (-1, 36, 5,  5)
        #self.conv2 = nn.Conv2d(1, 36, kernel_size=3, stride = 2)                # o/p_shape: (-1, 36, 5,  5)
        #self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=1)                   # o/p_shape: (-1, 36, 5,  5)

        #self.conv3 = nn.Conv2d(36, 64, kernel_size=2, stride =2)                # o/p_shape: (-1, 64, 2,  2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(36 * 5 * 5, 64)  # Adjusted based on the new feature dimensions
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 2)  # Output 2 value for binary classification

        self.to(device)  # Move the entire model to the device (GPU if available)

    def forward(self, x):

        x = torch.sigmoid(self.conv1(x))
        x = self.maxpool1(x)

        x = self.q_conv1(x * np.pi) # Converting features to Angles
        #print(x.shape)

        x = torch.relu(self.conv2(x))
        #x = self.maxpool2(x)

        #x = torch.relu(self.conv3(x))

        x = self.flatten(x)

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.log_softmax(self.fc3(x), dim=1)

        return x


In [ ]:
# Model, Loss, and Optimizer
model = HybridNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008)
# Training loop
epochs = 20
for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (feature, label) in tqdm(enumerate(train_loader)):
        feature, label = feature.to(device), label.to(device)
        print(label.requires_grad)
        optimizer.zero_grad()
        output = model(feature)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

0it [00:00, ?it/s]

False


0it [00:03, ?it/s]


KeyboardInterrupt: 

In [ ]:
X_test_tensor = torch.Tensor(X_test).view(-1, 1, input_dim[0], input_dim[1])
y_test_tensor = torch.FloatTensor(y_test)  # Convert to FloatTensor for binary classification
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images) #-1x2
        predictions = np.argmax(outputs.detach().cpu().numpy(), axis = 1)
        correct += np.sum(np.logical_not(np.logical_xor(predictions, labels.numpy())))
        total += len(predictions)

accuracy = correct / total
print(f'Test accuracy: {accuracy:.4f}')

Test accuracy: 0.4925


## Quanvolution - Not trained Just The Feature map

In [ ]:
class Quanvolution:
    def __init__(self, kernel_size, n_layers, stride):
        self.input = input
        self.kernel_size = kernel_size
        self.n_qubits = int(self.kernel_size**2)
        self.n_layers = n_layers
        self.stride = stride
        self.weight_shapes = {"weights": (self.n_layers, self.n_qubits)}
        self.dev = qml.device("default.qubit", wires = self.n_qubits)

        @qml.qnode(self.dev, interface="torch")
        def quantum_circuit(inputs, weights):

            # Hadamard Layer
            for wire in range(self.n_qubits):
                qml.Hadamard(wires = wire)

            # Embedding layer
            qml.AngleEmbedding(inputs, wires=range(self.n_qubits))
            '''
            # Variational layer
            for _ in range(self.n_layers):
                qml.StronglyEntanglingLayers(weights, wires=range(self.n_qubits))
            '''
            # Random Layer
            for _ in range(self.n_layers):
                qml.templates.RandomLayers(weights, wires=list(range(self.n_qubits)))

            # Measurement
            return [qml.expval(qml.PauliZ(wires=i)) for i in range(self.n_qubits)]

        self.qlayer = qml.qnn.TorchLayer(quantum_circuit, self.weight_shapes)

    def q_conv(self, input):
        # Assert the rules
        assert len(input.shape) == 4                                                # (batch_size, n_channels, x_dim, y_dim)
        assert input.shape[-1] == input.shape[-2]
        assert self.stride >= 0
        assert input.shape[1] == 1                                                  # Supports only one channel only

        k = self.kernel_size                                                             # kernel dimension
        output = None
        iterator = 0
        output_shape = int((input.shape[-1]-k)/self.stride + 1)
        #print(f"output_shape = {output_shape}\n")
        for i in range(0, input.shape[-2], self.stride):
            if(i+k > input.shape[-2]):
                break
            for j in range(0, input.shape[-1], self.stride):
                if(j+k > input.shape[-1]):
                    break
                x = torch.flatten(input[:,:, i:i+k, j:j+k], start_dim=1)
                #print(quantum_circuit(x)[0])
                exp_vals = self.qlayer(x).view(input.shape[0], self.n_qubits, 1)

                if(iterator>0):
                    output = torch.cat((output, exp_vals), -1)
                else:
                    output = exp_vals

                iterator +=1
        output = torch.reshape(output, (output.shape[0], output.shape[1], output_shape, output_shape))

        return output

    def __call__(self, input):
        return self.q_conv(input)

## Encode the features And save

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pennylane as qml
from pennylane import numpy as np
from tqdm import tqdm

# Quantum feature map

quanvolution = Quanvolution(kernel_size = 2, n_layers = 3, stride = 2)

In [ ]:
Q_features = quanvolution(torch.tensor(X, dtype=torch.float32))

In [ ]:
torch.save(Q_features, q_feature_file_path)
torch.save(torch.tensor(Y_bin), q_feature_labels_file_path)

## Define the Model for training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Define the Hybrid Neural Network

In [ ]:
# Create the Hybrid Model (Same as before)
class HybridNN(nn.Module):
    def __init__(self):
        super(HybridNN, self).__init__()
        # i/p_shape: (-1, 4, 32, 32)
        self.conv1 = nn.Conv2d(4, 32, kernel_size = 8, stride = 1)               # o/p_shape: (-1, 32 , 25, 25)

        self.maxpool1 = nn.MaxPool2d(kernel_size = 2, stride = 1)                # o/p_shape: (-1, 32 , 24, 24)

        self.conv2 = nn.Conv2d(32, 64, kernel_size = 8, stride = 1)                # o/p_shape: (-1, 64, 17,  17)
        self.maxpool2 = nn.MaxPool2d(kernel_size=4, stride=1)                   # o/p_shape: (-1, 64, 14, 14)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=8, stride =2)                # o/p_shape: (-1, 128, 4,  4)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 4 * 4, 128)  # Adjusted based on the new feature dimensions
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 8)
        self.fc4 = nn.Linear(8,2)# Output 2 value for binary classification

        #self.to(device)  # Move the entire model to the device (GPU if available)

    def forward(self, x):

        x = self.maxpool1(torch.relu(self.conv1(x)))
        x = self.maxpool2(torch.relu(self.conv2(x)))
        x = torch.relu(self.conv3(x))

        x = self.flatten(x)

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.log_softmax(self.fc4(x), dim=1)

        return x

In [ ]:
import pickle

# Function to save the results
def saveResults(results, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(results, f)

## K-fold cross validate

In [13]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
from Models import Hybrid_QuanvModel
from tqdm import tqdm
import pickle

'''
# load the quantum
Q_featureMap = torch.load(q_feature_file_path)
Q_labels = torch.load(q_feature_labels_file_path)
'''

# Model to test
#best_hybrid_model = hybridModel
criterion = torch.nn.CrossEntropyLoss()  # For classification tasks, adjust accordingly
num_epochs = 120

'''
# Convert your features and labels to PyTorch tensors
X = Q_featureMap
y = Q_labels

X.requires_grad = False
y.requires_grad = False
'''
X = torch.tensor(X, dtype = torch.float32)
y = torch.tensor(Y_bin, dtype = torch.float32)

# Define KFold cross validator
kf = KFold(n_splits=6, shuffle=True, random_state=42)
results_dict = {}
models = {}

# Define a device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loop through each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Fold {fold+1}")
    # Instantiating the hybrid model
    model = Hybrid_QuanvModel()
    model.to(device)
    model_name = f"HybridModel_Quanvolution_based_{fold+1}"
    models[model_name] = None
    results_dict[model_name] = {"loss": [], "test_acc": [], "precision": 0, "recall": 0, "F1_score": 0}
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Split the data into training and validation sets for this fold
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Convert labels to one-hot encoding
    y_train = torch.tensor(np.array(np.eye(2))[y_train.to(torch.int)], dtype=torch.float32)

    # Convert to PyTorch DataLoader
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=20, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=20)

    # Training loop for this fold
    for epoch in tqdm(range(num_epochs)):  # num_epochs is a variable you should define
        total_loss = 0
        avg_loss = 0
        for batch_idx, (feature, label) in tqdm(enumerate(train_loader)):
            feature, label = feature.to(device), label.to(device)
            optimizer.zero_grad()
            output = model(feature)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader.dataset)
        results_dict[model_name]["loss"].append(avg_loss)
        test_accuracy = test_acc(model, val_loader, device)
        results_dict[model_name]["test_acc"].append(test_accuracy)
    # Training done
    # Calculate the precision, recall and f1 score of this model
    true_labels = y_val.numpy()
    predictions = model(X_val.to(device))
    predicted_labels = np.argmax(predictions.detach().cpu().numpy(), axis = 1)
    results_dict[model_name]["precision"] = precision_score(true_labels, predicted_labels, average='macro')  # Use 'micro', 'macro', 'weighted', or 'samples' for multi-class
    results_dict[model_name]["recall"] = recall_score(true_labels, predicted_labels, average='macro')
    results_dict[model_name]["F1_score"] = f1_score(true_labels, predicted_labels, average='macro')
    #torch.save(model, f"RESULTS/HybridModels_k_fold_cross_val/{model_name}.pt")
    models[model_name] = model


# Save the results_dict
results_file_path = 'RESULTS/HybridModel_Quanvolution_k_fold_cross_val/hybridModel_Quanvolution_based_k-fold_cross_val_results_dictionary.pkl'
saveResults(results_dict, results_file_path)


Fold 1


  0%|          | 0/120 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [01:00, 60.60s/it]
2it [02:05, 63.10s/it]
3it [03:09, 63.60s/it]
4it [04:14, 64.13s/it]
5it [05:19, 64.53s/it]
6it [06:25, 64.77s/it]
7it [07:37, 65.31s/it]
  0%|          | 0/120 [07:37<?, ?it/s]


KeyboardInterrupt: 

## Testing

In [ ]:
model = HybridNN()
for p in model.named_parameters():
    print(p)

In [ ]:
import matplotlib.pyplot as plt

for model in results_dict.keys():
    results = results_dict[model]
    print(results["precision"])
    print(results["recall"])
    print(results["F1_score"])



0.23505434782608695
0.5
0.3197781885397412
0.22554347826086957
0.5
0.31086142322097376
0.2391304347826087
0.5
0.3235294117647059
0.22418478260869565
0.5
0.3095684803001876
0.24931880108991825
0.5
0.3327272727272727
0.22479564032697547
0.5
0.31015037593984957


In [ ]:
m = nn.LogSoftmax(dim=1)
input = torch.randn(2, 3)
m(input)

tensor([[-0.7614, -1.8229, -0.9904],
        [-0.7245, -2.0822, -0.9396]])

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm


# load the quantum
Q_featureMap = torch.load(q_feature_file_path)
Q_labels = torch.load(q_feature_labels_file_path)

# Model to test
#best_hybrid_model = hybridModel
criterion = torch.nn.CrossEntropyLoss()  # For classification tasks, adjust accordingly
num_epochs = 200

# Convert your features and labels to PyTorch tensors
X = Q_featureMap
y = Q_labels

X.requires_grad = False
y.requires_grad = False


# Define KFold cross validator
kf = KFold(n_splits=6, shuffle=True, random_state=42)
results_dict = {}
models = {}

# Define a device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = HybridNN()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0008)

# Split the data into training and validation sets for this fold
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = torch.tensor(np.array(np.eye(2))[y_train.to(torch.int)], dtype=torch.float32)

# Convert to PyTorch DataLoader
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64)

# Training loop for this fold
for epoch in range(num_epochs):  # num_epochs is a variable you should define
    total_loss = 0
    avg_loss = 0
    for batch_idx, (feature, label) in enumerate(train_loader):
        feature, label = feature.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(feature)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if(epoch%5 == 0):
        avg_loss = total_loss / len(train_loader.dataset)
        test_accuracy = test_acc(model, val_loader, device)
        print(f"test_acc: {test_accuracy}, loss: {avg_loss}")



test_acc: 0.5158371040723982, loss: 0.0111046877442574
test_acc: 0.5158371040723982, loss: 0.011008982992226305
test_acc: 0.5158371040723982, loss: 0.01100420755864271
test_acc: 0.4841628959276018, loss: 0.011003443882578895
test_acc: 0.4841628959276018, loss: 0.01100293568878217
test_acc: 0.4841628959276018, loss: 0.011003243544745066
test_acc: 0.4841628959276018, loss: 0.011002216682412457
test_acc: 0.4841628959276018, loss: 0.011002837192444574
test_acc: 0.4841628959276018, loss: 0.01100147932835447
test_acc: 0.4841628959276018, loss: 0.011002612492394826
test_acc: 0.4841628959276018, loss: 0.01100273065421046
test_acc: 0.4841628959276018, loss: 0.011002496425527024
test_acc: 0.4841628959276018, loss: 0.011002013066998955
test_acc: 0.4841628959276018, loss: 0.011002333931912099
test_acc: 0.4841628959276018, loss: 0.011003340959819266
test_acc: 0.4841628959276018, loss: 0.011002631549662203
test_acc: 0.4841628959276018, loss: 0.011003243544745066
test_acc: 0.4841628959276018, loss: 0

In [ ]:
from collections import Counter
Counter(y_train.numpy())

TypeError: unhashable type: 'numpy.ndarray'